# Toronto Neighborhoods
Author:    Guy Shavit  
### Part 1: scrape the Wikipedia page in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 

#### Importing packages

In [1]:
import pandas as pd

#### Get the Wikipedia page and read into a Dataframe

In [2]:
!wget -O List_of_postal_codes_of_Canada:_M https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
df = pd.read_html('List_of_postal_codes_of_Canada:_M')

--2021-02-12 20:01:02--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.153.224, 2620:0:860:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.153.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54740 (53K) [text/html]
Saving to: ‘List_of_postal_codes_of_Canada:_M’

List_of_postal_code 100%[===================>]  53.46K  --.-KB/s    in 0.003s  

2021-02-12 20:01:03 (17.5 MB/s) - ‘List_of_postal_codes_of_Canada:_M’ saved [54740/54740]



#### Take the first table from the Wikipedia page into a Dataframe, Drop any line with Boroguh "Not assigned", reset the index and set the Neighbourhood to the Borough name for any Neighbourhood that is "Not assigned"

In [3]:
PostCode = df[0]
PostCode.drop(PostCode.loc[PostCode['Borough']=="Not assigned"].index, inplace=True)
PostCode.reset_index(drop=True, inplace=True)
PostCode.loc[PostCode.Neighbourhood == "Not assigned", "Neighbourhood"] = PostCode.Borough
PostCode.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Get the table shape

In [4]:
PostCode.shape

(103, 3)

### Part 2:  Add latitude and the longitude coordinates of each neighborhood

#### Importing packages

In [5]:
!pip install geocoder
import geocoder


#### Create a list of latitudes and longitudes for each postal code 
#### lookup the latitude and longitude ia geocoder and add to the list.  
#### At the end add the lists to the dataframe

In [ ]:
latitude = []
longitude = []
for index, PC in PostCode.iterrows():
    # initialize lat_lng variable to None
    lat_lng_coords = None
    postal_code = PC['Postal Code']
    #loop until you get the coordinates
    while(lat_lng_coords is None):
          g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
          lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
PostCode['Latitude'] = latitude
PostCode['Longitude'] = longitude

In [ ]:
PostCode.head()

### Part 3:  Explore and cluster the neighborhoods in Toronto

#### Importing packages

In [ ]:
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library



#### Use geopy library to get the latitude and longitude values of Toronto.

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

#### Create a map of Toronto with neighborhoods superimposed on top.

In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(PostCode['Latitude'], PostCode['Longitude'], PostCode['Borough'], PostCode['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'ULAMQLZ3O2MJYTHU2QTK0QYRUZ3YP3ZNGOSFZ45PG2R4TQ0I' 
CLIENT_SECRET = 'Z5LBQMM1WEPQSJJWIQ3ALKHCVFLD4VR1DTSX1MCE0ZURWW3O' 
ACCESS_TOKEN = 'MZI1APQNRECXU5PRJZMY4BLZUHRGYZEZ24SU3NEYXMKWGV03' 

VERSION = '20200212' # Foursquare API version
LIMIT = 100 

#### function to get top 100 venus within 500m of a location

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    radius=500
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### run the function to get nearby venues for neighbourhood in Toronto

In [ ]:
toronto_venues = getNearbyVenues(names=PostCode['Neighbourhood'],
                                   latitudes=PostCode['Latitude'],
                                   longitudes=PostCode['Longitude']
                                  )

#### size and result dataframe

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

#### how many venues were returned for each neighborhood?

In [ ]:
toronto_venues.groupby('Neighbourhood').count()

#### How many of each catagory in each Neighbourhood?

In [ ]:
toronto_venues.groupby(['Neighbourhood','Venue Category']).count()

#### Which Neighbourhood has the most Chinese restaurants?

In [ ]:
toronto_venues[toronto_venues["Venue Category"] =='Chinese Restaurant'].groupby('Neighbourhood').count().sort_values(['Venue Category'], ascending=False).head(1)

### Analyze each Neighbourhood

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

And let's examine the new dataframe size.

In [ ]:
toronto_onehot.shape

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

#### Confirm the new size

In [ ]:
toronto_grouped.shape

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### put that into a panada dataframe

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

### Cluster the Neighbourhoods
#### Run _k_-means to cluster the neighborhood into 5 clusters.

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#toronto_merged = PostCode
toronto_merged = neighbourhoods_venues_sorted

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(PostCode.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.groupby('Cluster Labels').head()

#### Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 2

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 3

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 4

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 5

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]